# Intro
While this notebook focuses on understanding through clear explanations and visualizations,a professional implementation could involve separate files for better organization:
- **model.py**: Contains the neural network architecture.
- **agent.py**: Contains the DRL algorithm.
- **trainning.py**: Contains the agent interaction with the environment to provide the training and testing loops.

This notebook contains all above files in one place for better understanding! Consider this notebook the first chapter of our DRL adventure! In next notebooks, we'll be more focused on the implementation of the algorithms. 


# Simulation Environment

In [1]:
import gymnasium as gym
import ale_py

# Create the Pong environment
env = gym.make("ALE/Pong-v5")

# Reset the environment to start
state = env.reset()

# Example interaction with the environment
for _ in range(100):
    action = env.action_space.sample()  # Take a random action
    observation, _, terminated, truncated, done = env.step(action)  # Apply the action

    if done:
        state = env.reset()  # Reset the environment if done

print(done)


A.L.E: Arcade Learning Environment (version 0.9.0+750d7f9)
[Powered by Stella]


{'lives': 0, 'episode_frame_number': 4, 'frame_number': 400}
